In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
test_data = {}
# blocks 1 and 3 are gain and have 4 for win and 3 for loss
# blocks 2 and 4 are loss and have 2 for win and 1 for loss
for i in range(1,101):
    testData = spio.loadmat('genData_smG_rlG_stickyActG_stickyStimGL_wslsG_hG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    block = [
                {
                    'action': np.array([[resp-1 for ([id],[resp]) in zip(struct[0,0]['blockID'],struct[0,0]['resp']) if id==blockID]]),
                    'state': np.array([[stimOffers for ([id],stimOffers) in zip(struct[0,0]['blockID'],struct[0,0]['stimOffers']) if id==blockID]]),
                    'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew']) if id==blockID]]),
                    'id': 'S'+str(i),

                    'block': blockID
                } for blockID in np.unique([struct[0,0]['blockID']])

            ]
#     block = [
#                 {
#                     'action': np.array([struct[0,0]['resp'].flatten()])-1,
#                     'state': np.array([struct[0,0]['stimOffers']]),
#                     'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew'])]]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
    if i<91:
        data['S'+str(i)] = block
    else:
        test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
           1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
           0, 0, 1, 0]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],
           [1, 2],
           [2, 1],
           [3, 2],
           [3, 2],
           [2, 3],
           [3, 1],
           [2, 3],
           [1, 3],
           [3, 2],
           [2, 1],
           [2, 1],
           [3, 2],
           [1, 3],
           [2, 1],
           [2, 1],
           [2, 3],
           [1, 2],
           [2, 3],
           [3, 2],
           [1, 3],
           [3, 1],
           [3, 2],
           [3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [2, 3

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/fullModel_rewardOutcome/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-02 14:29:01,333 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:29:01,334 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:29:01,336 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:29:01,337 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:29:01,338 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-02 14:29:23,293 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/fullModel_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:29:57,664 - DeepRL - DEBUG - global iter =    0 total obj: 11618.2767
2021-07-02 14:30:20,492 - DeepRL - DEBUG - global iter =    1 total obj: 10272.0355
2021-07-02 14:30:43,518 - DeepRL - DEBUG - global iter =    2 total obj: 10152.0845
2021-07-02 14:31:08,109 - DeepRL - DEBUG - global iter =    3 total obj: 10118.2259
2021-07-02 14:31:35,026 - DeepRL - DEBUG - global iter =    4 total obj: 10104.1480
2021-07-02 14:32:03,922 - DeepRL - DEBUG - global iter =    5 total obj: 10071.4528
2021-07-02 14:32:32,405 - DeepRL - DEBUG - global iter =    6 total obj: 10066.8567
2021-07-02 14:33:00,463 - DeepRL - DEBUG - global iter =    7 total obj: 10063.5664
2021-07-02 14:33:27,588 - DeepRL - DEBUG - global iter =    8 total obj: 10051.4908
2021-07-02 14:33:55,635 - DeepRL - DEBUG - global iter =    9 total obj: 10030.4816
2021-07-02 14:34:23,855 - DeepRL - DEBUG - global iter =   10 total obj: 10053.8956
2021-07-02 14:34:51,726 - DeepRL - DEBUG - global iter =   11 total obj: 100

INFO:tensorflow:../results/fullModel_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[4, 3, 4, ..., 4, 4, 4],
          [2, 1, 2, ..., 2, 2, 2],
          [4, 3, 4, ..., 3, 3, 4],
          ...,
          [2, 1, 2, ..., 2, 2, 2],
          [4, 4, 4, ..., 3, 4, 3],
          [2, 2, 2, ..., 1, 2, 1]], dtype=int32),
   'action': array([[0, 1, 0, ..., 0, 1, 0],
          [0, 1, 0, ..., 0, 1, 0],
          [0, 1, 0, ..., 1, 0, 1],
          ...,
          [0, 1, 0, ..., 0, 1, 0],
          [0, 0, 0, ..., 1, 1, 0],
          [0, 0, 0, ..., 1, 1, 0]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[1, 2],
           [3, 1],
           [1, 3],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          ...,
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ..

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-02 14:49:10,372 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:49:10,374 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:49:10,375 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:49:10,376 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:49:10,377 - DeepRL - DEBUG - test data-points: 10
2021-07-02 14:49:31,397 - DeepRL - DEBUG - opt started...
2021-07-02 14:49:31,398 - DeepRL - DEBUG - started testing...
2021-07-02 14:51:30,814 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:51:50,243 - DeepRL - DEBUG - global iter =    0 total obj: 11676.6719
2021-07-02 14:52:02,086 - DeepRL - DEBUG - global iter =    1 total obj: 10435.8545
2021-07-02 14:52:13,847 - DeepRL - DEBUG - global iter =    2 total obj: 10198.5187
2021-07-02 14:52:25,820 - DeepRL - DEBUG - global iter =    3 total obj: 10155.4571
2021-07-02 14:52:40,601 - DeepRL - DEBUG - global iter =    4 total obj: 10174.6800
2021-07-02 14:52:57,081 - DeepRL - DEBUG - global iter =    5 total obj: 10116.2418
2021-07-02 14:53:14,921 - DeepRL - DEBUG - global iter =    6 total obj: 10084.7883
2021-07-02 14:53:32,269 - DeepRL - DEBUG - global iter =    7 total obj: 10075.4121
2021-07-02 14:53:49,218 - DeepRL - DEBUG - global iter =    8 total obj: 10066.7662
2021-07-02 14:54:08,708 - DeepRL - DEBUG - global iter =    9 total obj: 10058.6004
2021-07-02 14:54:08,710 - DeepRL - DEBUG - started testing...
2021-07-02 14:56:19,359 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_rewardOutcome/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:56:36,399 - DeepRL - DEBUG - global iter =   10 total obj: 10052.0023
2021-07-02 14:56:48,136 - DeepRL - DEBUG - global iter =   11 total obj: 10031.5065
2021-07-02 14:56:59,975 - DeepRL - DEBUG - global iter =   12 total obj: 10047.3253
2021-07-02 14:57:13,906 - DeepRL - DEBUG - global iter =   13 total obj: 10036.3657
2021-07-02 14:57:28,697 - DeepRL - DEBUG - global iter =   14 total obj: 10057.8705
2021-07-02 14:57:45,454 - DeepRL - DEBUG - global iter =   15 total obj: 10035.3959
2021-07-02 14:58:02,472 - DeepRL - DEBUG - global iter =   16 total obj: 10016.6059
2021-07-02 14:58:19,816 - DeepRL - DEBUG - global iter =   17 total obj: 10021.6891
2021-07-02 14:58:35,970 - DeepRL - DEBUG - global iter =   18 total obj: 10010.2241
2021-07-02 14:58:50,630 - DeepRL - DEBUG - global iter =   19 total obj: 10016.2334
2021-07-02 14:58:50,632 - DeepRL - DEBUG - started testing...
2021-07-02 15:00:57,175 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/fullModel_rewardOutcome/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:05:43,298 - DeepRL - DEBUG - global iter =   30 total obj: 9945.8275
2021-07-02 15:05:57,863 - DeepRL - DEBUG - global iter =   31 total obj: 9960.1950
2021-07-02 15:06:12,049 - DeepRL - DEBUG - global iter =   32 total obj: 9924.2504
2021-07-02 15:06:24,472 - DeepRL - DEBUG - global iter =   33 total obj: 9896.7788
2021-07-02 15:06:39,228 - DeepRL - DEBUG - global iter =   34 total obj: 9898.7992
2021-07-02 15:06:53,449 - DeepRL - DEBUG - global iter =   35 total obj: 9902.6704
2021-07-02 15:07:07,848 - DeepRL - DEBUG - global iter =   36 total obj: 9888.4968
2021-07-02 15:07:21,340 - DeepRL - DEBUG - global iter =   37 total obj: 9870.6697
2021-07-02 15:07:32,256 - DeepRL - DEBUG - global iter =   38 total obj: 9892.2489
2021-07-02 15:07:43,563 - DeepRL - DEBUG - global iter =   39 total obj: 9853.2642
2021-07-02 15:07:43,565 - DeepRL - DEBUG - started testing...
2021-07-02 15:09:38,784 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_rewardOutcome/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:10:12,691 - DeepRL - DEBUG - global iter =   40 total obj: 9849.0562
2021-07-02 15:10:26,878 - DeepRL - DEBUG - global iter =   41 total obj: 9840.6181
2021-07-02 15:10:39,581 - DeepRL - DEBUG - global iter =   42 total obj: 9839.4009
2021-07-02 15:10:50,866 - DeepRL - DEBUG - global iter =   43 total obj: 9875.6360
2021-07-02 15:11:02,071 - DeepRL - DEBUG - global iter =   44 total obj: 9829.4670
2021-07-02 15:11:15,375 - DeepRL - DEBUG - global iter =   45 total obj: 9829.4694
2021-07-02 15:11:29,020 - DeepRL - DEBUG - global iter =   46 total obj: 9826.7280
2021-07-02 15:11:40,290 - DeepRL - DEBUG - global iter =   47 total obj: 9866.9735
2021-07-02 15:11:49,745 - DeepRL - DEBUG - global iter =   48 total obj: 9861.7470
2021-07-02 15:11:59,104 - DeepRL - DEBUG - global iter =   49 total obj: 9844.0358
2021-07-02 15:11:59,106 - DeepRL - DEBUG - opt finished.
2021-07-02 15:11:59,107 - DeepRL - DEBUG - started testing...
2021-07-02 15:12:14,714 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/fullModel_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
          0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
          1, 1, 1, 1]]),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [2, 1],
          [1, 3],
          [2, 1],
          [2, 1],
          [1, 3],
          [3, 2],
          [3, 1],
          [1, 3],
          [2, 3],
          [1, 2],
          [1, 3],
          [2, 1],
          [1, 3],
          [3, 1],
          [1, 2],
          [2, 3],
          [2, 1],
          [3, 1],
          [2, 1],
          [3, 2],
          [1, 2],
          [1, 2],
          [2, 3],
          [1, 3],
          [3, 2],
          [2, 3],
          [3, 1],
          [3, 1],
          [2